In [1]:
import os
import datetime
import json
import re
import copy
from scipy.sparse import csr_matrix, lil_matrix, eye, csgraph
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.sparse
from scipy.stats import norm, pearsonr
import matplotlib.pyplot as plt
from muskingum import MuskingumCunge, KalmanFilter
from muskingum import _ax, _ax_bu, _apply_gain
from muskingum import numba_init_inflows, numba_matmat, numba_matmat_par
import time
from numba import njit
import seaborn as sns
import geopandas as gpd
import matplotlib as mpl
from simulation_util import read_csv, load_nwm_data, prepare_inputs, update_model_inputs, save_raw_data, simulate_open, simulate_model_with_callbacks
from tqdm import tqdm
import asyncio
import pickle

In [2]:
basepath = '/Users/jeiloh/Desktop/forecasting_overtopping/guadalupe/data'
nwm_export = 'export_Guadalupe_2023'
nwm_vars = ['qBucket', 'qSfcLatRunoff', 'streamflow']
date_list = pd.date_range('20230401', '20230701', freq='h', tz='utc')[:-1]
multi_models = [f'short_range_{i}' for i in range(1, 12)]
alpha = np.linspace(0, 2, len(multi_models))

In [3]:
streamflow = pd.read_csv(f'{basepath}/{nwm_export}/2023040100/2023040100_short_range_streamflow.csv')
runoff = pd.read_csv(f'{basepath}/{nwm_export}/2023040100/2023040100_short_range_qSfcLatRunoff.csv')
bucket = pd.read_csv(f'{basepath}/{nwm_export}/2023040100/2023040100_short_range_qBucket.csv')
usgs = pd.read_csv('usgs_gages_nudging.csv')

In [4]:
streamflow['time'] = pd.to_datetime(streamflow['time'])
streamflow = streamflow.set_index('time').tz_localize('UTC')
usgs.set_index('datetime', inplace=True)
usgs.index = pd.to_datetime(usgs.index).tz_convert('UTC')
usgs = usgs[usgs.index.isin(pd.date_range('20230401', '20230801', freq='h', tz='utc'))]

inputs = runoff + bucket

In [5]:
with open('/Users/jeiloh/Desktop/forecasting_overtopping/guadalupe/data/gua_together_cleaned.json') as texas:
    d = json.load(texas)

In [6]:
comids = [5256723, 5256725, 5280902, 5293763, 5275764, 9535875, 9535873, 8333624, 8333590, 1077643, 
          1078881, 1078897, 1477505, 1158941, 1160619, 1160719, 1111369, 1166621, 1142744, 1143298, 
          1145428, 1113965, 1166607, 1306945, 1269780, 1279648, 1287711, 1264655, 1262905, 1294198, 
          1262851, 1300456, 1300458, 4133145, 1448010, 1456345, 1456323, 1494050, 1515495, 1523581, 
          1469612, 1508325, 1468428, 1468426, 1475693, 1475695, 1475697, 1479839, 1523627, 1515213, 
          1515205, 3123150, 3123710, 13514834, 13494837, 13700659, 14034073, 13681329, 14034081, 
          14034505, 5544034, 5487027, 5507108, 5521348, 5525867, 5499448, 5499450, 5514792, 
          5514790, 5523992, 5531600, 5531596, 5559336, 5559442, 5570839, 5579047, 1572614, 1572640, 
          1576430, 2572780, 2572328, 5588526, 5593582, 5671495, 5635929, 5651989, 5652493, 5714895, 
          5690898, 5714893, 5715109, 5722371, 5722367, 5656548, 5702833, 5702865, 5712877, 5718726,
          5729263, 5746592, 5737005, 5735553, 5746512, 5746014, 5754392, 5757862, 5762659, 5765169,
          5771837, 5789878, 5780397, 5791934, 3766328, 9354366, 1609042, 1612200, 1615408, 7847707, 
          3589578, 1623369, 1623367, 1632017, 1640215, 10840568, 10840560, 10836114, 7852413, 9354358, 
          9357548, 9351103, 9351055, 3154805, 7872307, 7876612, 10630401, 10624041, 10631455, 942110011, 
          10648495, 10661028, 10657909, 10648627, 10657911, 10664258, 10671447, 10828988]

In [20]:
GAUGES_USGS = {
    # san antonio
    "10840818":'USGS-08178050',
    "10840824":'USGS-08178500',
    "10840502":'USGS-08178565',
    "10840230":'USGS-08178593',
    "10840232":'USGS-08178700',
    "10840488":'USGS-08178800',
    "10834470":'USGS-0817887350', 
    "10833740":'USGS-08178880',
    "10834916":'USGS-08178980', 
    "10835018":'USGS-08180586', 
    "10836382":'USGS-08180640', 
    "10835982":'USGS-08180700',
    "10836104":'USGS-08180800', 
    "10835030":'USGS-08181400', 
    "10836388":'USGS-08181480',
    "10836092":'USGS-08181500', 
    "10840558":'USGS-08181725',
    "10840572":'USGS-08181800',
    "3836053":'USGS-08183200',
    "3838221":'USGS-08183500', 
    "7850579":'USGS-08183900', 
    "7850611":'USGS-08183978', 
    "7850687":'USGS-08185065', 
    "7851629":'USGS-08185100', 
    "7851771":'USGS-08185500',
    "7852265":'USGS-08186000',
    "3838999":'USGS-08186500',
    "3839263":'USGS-08187500',
    "3839167":'USGS-08188060', 
    "3840125":'USGS-08188500',
    "3840137":'USGS-08188570',
    #Guadalupe
    "3585678":"USGS-08165300",
    "3585620":"USGS-08165500",
    "3585554":"USGS-08166000",
    "3585626":"USGS-08166140",
    "3585724":"USGS-08166200",
    "3587616":"USGS-08166250",
    "3589508":"USGS-08167000",
    "3589062":"USGS-08167200",
    "3589120":"USGS-08167500",
    "1619595":"USGS-08167800",
    "1619637":"USGS-08168000",
    "1620031":"USGS-08168500",
    "1619663":"USGS-08168797",
    "1619647":"USGS-08168932", 
    "1619649":"USGS-08169000",
    "1620877":"USGS-08169792",
    "1622735":"USGS-08169845",
    "1631099":"USGS-08170500",
    "1628253":"USGS-08170950",
    "1628219":"USGS-08170990",
    "1630223":"USGS-08171000",
    "1629555":"USGS-08171290",
    "1631129":"USGS-08171350",
    "1631195":"USGS-08171400",
    "1631387":"USGS-08172000",
    "1631087":"USGS-08172400",
    "1631587":"USGS-08173000",
    "1622713":"USGS-08173900", 
    "1620735":"USGS-08174200",
    "1620703":"USGS-08174550",
    "1622763":"USGS-08174600",
    "1623075":"USGS-08174700",
    "1623207":"USGS-08175000",
    "1637437":"USGS-08175800",
    "1639225":"USGS-08176500",
    "1638559":"USGS-08176900",
    "1638907":"USGS-08177500",
}


In [8]:
rename_dict = {v.split('-')[-1]: k for k, v in GAUGES_USGS.items() if v.split('-')[-1] in usgs.columns}
usgs = usgs.rename(columns=rename_dict)
usgs_ = [int(key) for key in GAUGES_USGS.keys()]
comids = [str(comid) for comid in usgs_]

In [9]:
link_ids = [i['attributes']['COMID'] for i in d['features']]
reach_ids = [str(x) for x in link_ids]
site_indices = {idx: reach_ids.index(idx) for idx in sorted(comids) if idx in reach_ids}


In [10]:
mc = MuskingumCunge(d, json_type='nhd', create_state_space=False, sparse=True, verbose=False)
KX = pd.read_csv('KX.csv')
K = KX.iloc[0]
X = KX.iloc[1]
mc.K[:] = K
mc.X[:] = X
mc.compute_muskingum_coeffs()

In [11]:
p_t = inputs
p_t.columns = p_t.columns.astype(str)
p_t = p_t[mc.reach_ids]
streamflow.columns = streamflow.columns.astype(str)
streamflow = streamflow[mc.reach_ids]

mc.datetime = streamflow.index[0] - pd.to_timedelta('1h')
mc.save_state()

In [12]:
mc.o_t_prev = streamflow.iloc[0][mc.reach_ids].fillna(0.).values
mc.o_t_next = streamflow.iloc[0][mc.reach_ids].fillna(0.).values
mc.i_t_next = 0 * streamflow.iloc[0][mc.reach_ids].fillna(0.).values
np.add.at(mc.i_t_next, mc.endnodes, mc.o_t_next[mc.endnodes])

ix = np.asarray(list(site_indices.values()), dtype=int)
ix.sort()
model_collection = mc.split(ix, p_t, create_state_space=False)
model_collection.save_states()


In [13]:
raw = save_raw_data(date_list, basepath, nwm_export, nwm_vars, mc)

Saving raw data: 100%|██████████| 2184/2184 [18:26<00:00,  1.97it/s]


In [14]:
with open('raw_data.pkl', 'wb') as file:
    pickle.dump(raw, file)

In [15]:
all_outputs_open = await simulate_open(date_list, basepath, nwm_export, nwm_vars, mc, model_collection)

Simulating open model: 100%|██████████| 2184/2184 [19:14<00:00,  1.89it/s]


In [16]:
with open('open_data.pkl', 'wb') as file:
    pickle.dump(all_outputs_open, file)

In [13]:
multi_model_collection_da = {}
for model in multi_models:
    multi_model_collection_da[model] = copy.deepcopy(model_collection)

In [14]:
for model_name in multi_models:
    for key, model in multi_model_collection_da[model_name].models.items():
        model_sites = [reach_id for reach_id in model.reach_ids if reach_id in usgs.columns]
        if model_sites:
            measurements = usgs[[reach_id for reach_id in model.reach_ids if reach_id in usgs.columns]]
            Q_cov = 2 * np.eye(model.n)
            R_cov = 1e-2 * np.eye(measurements.shape[1])
            kf = KalmanFilter(model, measurements, Q_cov, R_cov)
            if not hasattr(model, 'callbacks'):
                model.callbacks = {}
            model.callbacks['kf'] = kf
            assert model.o_t_next[kf.s].size == kf.measurements.columns.size

In [15]:
all_model_outputs = {}
for i, model_name in enumerate(multi_models):
    print(f"Simulating model: {model_name}")
    alpha_value = alpha[i]
    outputs = await simulate_model_with_callbacks(
        date_list=date_list,
        basepath=basepath,
        nwm_export=nwm_export,
        nwm_vars=nwm_vars,
        model_collection_da=multi_model_collection_da,
        model_name=model_name,
        alpha=alpha_value,
        usgs=usgs
    )
    all_model_outputs[model_name] = outputs
    with open(f'{model_name}_outputs.pkl', 'wb') as file:
        pickle.dump(all_model_outputs[model_name], file)

Simulating model: short_range_1


Simulating model short_range_1: 100%|██████████| 2184/2184 [25:08<00:00,  1.45it/s]


Simulating model: short_range_2


Simulating model short_range_2: 100%|██████████| 2184/2184 [25:13<00:00,  1.44it/s]


Simulating model: short_range_3


...

Simulating model short_range_9: 100%|██████████| 2184/2184 [25:47<00:00,  1.41it/s]


Simulating model: short_range_10


Simulating model short_range_10: 100%|██████████| 2184/2184 [25:38<00:00,  1.42it/s]


Simulating model: short_range_11


Simulating model short_range_11: 100%|██████████| 2184/2184 [25:30<00:00,  1.43it/s] 
